In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.datasets import imdb
import keras
from keras.utils import to_categorical
from sklearn.metrics import f1_score

BASE_DIR = ''
GLOVE_DIR = '/home/abhinav/data/GLOVE_DATA/glove.6B'
TEXT_DATA_DIR = '/home/abhinav/data/full_text'
THIRD_MOD_DIR = "/home/abhinav/data/third_model/"
SECOND_MOD_DIR = "/home/abhinav/data/second_model/"
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 200
FOURTH_MOD_DIR = "/home/abhinav/data/fourth_model/"

print('Loading data...')
[x_train, y_train] = pd.read_pickle(FOURTH_MOD_DIR + "train_data_10ratio") 

[x_test, y_test] = pd.read_pickle(FOURTH_MOD_DIR + "test_dataset") 

[x_val, y_val ] = pd.read_pickle(FOURTH_MOD_DIR + "validation_dataset")

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D,merge, MaxPooling1D, Embedding,Merge,  Dropout
from keras.models import Model
from keras.models import Sequential
from keras.layers.merge import Add,Concatenate,Dot

num_words = MAX_NB_WORDS

embedding_matrix = pd.read_pickle("/home/abhinav/data/fourth_model/embedding_matrix_for_selected_words")

Using TensorFlow backend.


Loading data...


In [5]:
from keras import backend as K
import os

def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend

set_keras_backend("theano")

In [16]:
class Metrics(keras.callbacks.Callback):
    def __init__(self,filepath):
        self.filepath = filepath
        self.best = -np.Inf
    def on_epoch_end(self, epoch,batch, logs={}):
        predict = self.model.predict(self.validation_data[0],batch_size = 512)
        predict = prob_to_label(predict)
        targ = self.validation_data[1]
        self.f1s=f1(targ, predict)
        """if self.f1s > self.best:
            self.best = self.f1s
            print('Epoch %05d: improved f1 to %0.5f,'
                                  ' saving model to %s'
                                  % (epoch, self.best, self.filepath))
            self.model.save(self.filepath,overwrite =True)"""
        print ("\nWeighted F1 score found on Validation dataset : " ,self.f1s)
        return

from sklearn.metrics import precision_recall_fscore_support
def f1(y_true, y_pred):
    return precision_recall_fscore_support(y_true = y_true, y_pred = y_pred)
ndm = keras.optimizers.Nadam(lr = 0.001)    

def prob_to_label(array):
    for a in array:
        if a[0] > a[1] :
            a[0] = 1
            a[1] = 0
        elif a[0] < a[1]:
            a[0] = 0
            a[1] = 1
        else:
            ind = np.random.randint(2)
            a[ind]= 1
            a[int(1-ind)] = 0
    return array

In [36]:
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers
from keras.layers import Bidirectional, LSTM, GRU,TimeDistributed

In [6]:

class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.RandomNormal()
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        eij = K.tanh(K.dot(x, self.W))
        
        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])


In [21]:

embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_gru = Bidirectional(GRU(50, return_sequences=True))(embedded_sequences)
l_att = AttLayer()(l_gru)
preds = Dense(2, activation='softmax')(l_att)
model = Model(sequence_input, preds)

In [41]:
MAX_SENTS  = 15

In [45]:

sentence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

review_input = Input(shape=(MAX_SENTS,MAX_SEQUENCE_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(LSTM(100))(review_encoder)
preds = Dense(2, activation='softmax')(l_lstm_sent)
model = Model(review_input, preds)


In [42]:
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_dense = TimeDistributed(Dense(200))(l_lstm)
l_att = AttLayer()(l_dense)
sentEncoder = Model(sequence_input, l_att)

review_input = Input(shape=(MAX_SENTS,MAX_SEQUENCE_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)

l_dense_sent = TimeDistributed(Dense(200))(l_lstm_sent)
l_att_sent = AttLayer()(l_dense_sent)
preds = Dense(2, activation='softmax')(l_att_sent)
model = Model(review_input, preds)

/home/abhinav/miniconda2/envs/tensorflow/lib/python2.7/site-packages/keras/engine/topology.py:2248: UserWarning: Class `__main__.AttLayer` defines `get_output_shape_for` but does not override `compute_output_shape`. If this is a Keras 1 layer, please implement `compute_output_shape` to support Keras 2.
  shapes = _to_list(layer.compute_output_shape(computed_tensors[0]._keras_shape))


ValueError: Input 0 is incompatible with layer time_distributed_6: expected ndim=3, found ndim=4

In [29]:
del(model)

In [34]:
model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, weights=[embedding_matrix], 
                    input_length=MAX_SEQUENCE_LENGTH,trainable = False))
model.add(Bidirectional(LSTM(50,return_sequences=True)))
model.add(Dropout(0.5))
model.add(AttLayer())
model.add(Flatten())
model.add(Dense(2, activation='softmax'))


ValueError: Input 0 is incompatible with layer flatten_2: expected min_ndim=3, found ndim=2

In [46]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 15, 1000)          0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 15, 200)           10240800  
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 200)               240800    
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 402       
Total params: 10,482,002
Trainable params: 10,482,002
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])

In [49]:
#model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

ValueError: Error when checking input: expected input_15 to have 3 dimensions, but got array with shape (13150, 1000)

In [10]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 10},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13056/13150 [============================>.] - ETA: 1s - loss: 1.0621 - acc: 0.6901
Weighted F1 score found on Validation dataset :  (array([ 0.9995951 ,  0.03045781]), array([ 0.65833333,  0.97575758]), array([ 0.7938422 ,  0.05907173]), array([15000,   165]))
13150/13150 [==============================] - 325s - loss: 1.0628 - acc: 0.6911 - val_loss: 0.6460 - val_acc: 0.6618
Epoch 2/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.6110 - acc: 0.8635
Weighted F1 score found on Validation dataset :  (array([ 0.99835858,  0.08115778]), array([ 0.89206667,  0.86666667]), array([ 0.94222441,  0.14841723]), array([15000,   165]))
13150/13150 [==============================] - 327s - loss: 0.6098 - acc: 0.8636 - val_loss: 0.2627 - val_acc: 0.8918
Epoch 3/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.4403 - acc: 0.8929
Weighted F1 score found on Validation dataset :  (array([ 0.99821681,  0.

In [11]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13056/13150 [============================>.] - ETA: 1s - loss: 1.0244 - acc: 0.7000
Weighted F1 score found on Validation dataset :  (array([ 0.99831275,  0.06726246]), array([ 0.8678    ,  0.86666667]), array([ 0.92849246,  0.12483632]), array([15000,   165]))
13150/13150 [==============================] - 368s - loss: 1.0207 - acc: 0.7016 - val_loss: 0.3410 - val_acc: 0.8678
Epoch 2/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.6201 - acc: 0.8545
Weighted F1 score found on Validation dataset :  (array([ 0.99915154,  0.04587156]), array([ 0.78506667,  0.93939394]), array([ 0.87926529,  0.08747178]), array([15000,   165]))
13150/13150 [==============================] - 325s - loss: 0.6192 - acc: 0.8541 - val_loss: 0.4175 - val_acc: 0.7867
Epoch 3/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.4360 - acc: 0.8955
Weighted F1 score found on Validation dataset :  (array([ 0.99868523,  0.

In [12]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 15},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13056/13150 [============================>.] - ETA: 1s - loss: 1.0880 - acc: 0.6324
Weighted F1 score found on Validation dataset :  (array([ 0.99876594,  0.04983389]), array([ 0.80933333,  0.90909091]), array([ 0.89412631,  0.09448819]), array([15000,   165]))
13150/13150 [==============================] - 317s - loss: 1.0859 - acc: 0.6333 - val_loss: 0.4298 - val_acc: 0.8104
Epoch 2/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.7124 - acc: 0.8229
Weighted F1 score found on Validation dataset :  (array([ 0.99730674,  0.08969867]), array([ 0.9134    ,  0.77575758]), array([ 0.95351103,  0.16080402]), array([15000,   165]))
13150/13150 [==============================] - 316s - loss: 0.7101 - acc: 0.8236 - val_loss: 0.2316 - val_acc: 0.9119
Epoch 3/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.5304 - acc: 0.8696
Weighted F1 score found on Validation dataset :  (array([ 0.99853112,  0.

In [13]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 20},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13056/13150 [============================>.] - ETA: 1s - loss: 1.3914 - acc: 0.3807
Weighted F1 score found on Validation dataset :  (array([ 0.999571  ,  0.02756377]), array([ 0.62133333,  0.97575758]), array([ 0.76632133,  0.05361305]), array([15000,   165]))
13150/13150 [==============================] - 322s - loss: 1.3917 - acc: 0.3814 - val_loss: 0.8202 - val_acc: 0.6252
Epoch 2/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.8743 - acc: 0.7926
Weighted F1 score found on Validation dataset :  (array([ 0.99938747,  0.0422799 ]), array([ 0.7614    ,  0.95757576]), array([ 0.86431058,  0.08098411]), array([15000,   165]))
13150/13150 [==============================] - 321s - loss: 0.8744 - acc: 0.7924 - val_loss: 0.5052 - val_acc: 0.7635
Epoch 3/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.6378 - acc: 0.8548
Weighted F1 score found on Validation dataset :  (array([ 0.99919692,  0.

In [14]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 30},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13056/13150 [============================>.] - ETA: 1s - loss: 1.5838 - acc: 0.4036
Weighted F1 score found on Validation dataset :  (array([ 0.99924573,  0.0482524 ]), array([ 0.79486667,  0.94545455]), array([ 0.88541512,  0.09181872]), array([15000,   165]))
13150/13150 [==============================] - 320s - loss: 1.5791 - acc: 0.4064 - val_loss: 0.5419 - val_acc: 0.7965
Epoch 2/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.9673 - acc: 0.7640
Weighted F1 score found on Validation dataset :  (array([ 0.99930957,  0.04387926]), array([ 0.77193333,  0.95151515]), array([ 0.87102719,  0.08388993]), array([15000,   165]))
13150/13150 [==============================] - 320s - loss: 0.9671 - acc: 0.7647 - val_loss: 0.5668 - val_acc: 0.7739
Epoch 3/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.6484 - acc: 0.8555
Weighted F1 score found on Validation dataset :  (array([ 0.99766679,  0.

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=15,
          class_weight = {0. : 1, 1. : 50},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=15,
          class_weight = {0. : 1, 1. : 75},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 100},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 125},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [17]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 150},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/20
13056/13150 [============================>.] - ETA: 1s - loss: 2.7308 - acc: 0.1568
Weighted F1 score found on Validation dataset :  (array([ 1.        ,  0.01704545]), array([ 0.36566667,  1.        ]), array([ 0.53551379,  0.03351955]), array([15000,   165]))
13150/13150 [==============================] - 321s - loss: 2.7281 - acc: 0.1587 - val_loss: 1.2697 - val_acc: 0.3726
Epoch 2/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.8886 - acc: 0.5123
Weighted F1 score found on Validation dataset :  (array([ 0.99960684,  0.03225806]), array([ 0.678     ,  0.97575758]), array([ 0.80797648,  0.06245151]), array([15000,   165]))
13150/13150 [==============================] - 319s - loss: 1.8832 - acc: 0.5138 - val_loss: 0.8559 - val_acc: 0.6812
Epoch 3/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.3161 - acc: 0.7084
Weighted F1 score found on Validation dataset :  (array([ 0.9993245 ,  0.

KeyboardInterrupt: 

In [16]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 200},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/20
13056/13150 [============================>.] - ETA: 1s - loss: 3.3969 - acc: 0.0895
Weighted F1 score found on Validation dataset :  (array([ 0.        ,  0.01088032]), array([ 0.,  1.]), array([ 0.        ,  0.02152642]), array([15000,   165]))
13150/13150 [==============================] - 328s - loss: 3.3978 - acc: 0.0898 - val_loss: 2.6060 - val_acc: 0.0109
Epoch 2/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.9941 - acc: 0.3591
Weighted F1 score found on Validation dataset :  (array([ 1.        ,  0.01807427]), array([ 0.4024,  1.    ]), array([ 0.57387336,  0.03550678]), array([15000,   165]))
13150/13150 [==============================] - 331s - loss: 1.9972 - acc: 0.3592 - val_loss: 1.7261 - val_acc: 0.4089
Epoch 3/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.2913 - acc: 0.7218
Weighted F1 score found on Validation dataset :  (array([ 0.9995438 ,  0.02516805]), array([ 0.58

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 500},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [15]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 1000},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/20
13056/13150 [============================>.] - ETA: 1s - loss: 8.1149 - acc: 0.0897
Weighted F1 score found on Validation dataset :  (array([ 0.        ,  0.01088032]), array([ 0.,  1.]), array([ 0.        ,  0.02152642]), array([15000,   165]))
13150/13150 [==============================] - 325s - loss: 8.0891 - acc: 0.0895 - val_loss: 3.4302 - val_acc: 0.0109

/home/abhinav/miniconda2/envs/tensorflow/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Epoch 2/20
 7168/13150 [===============>..............] - ETA: 86s - loss: 3.4647 - acc: 0.0871

KeyboardInterrupt: 